# 2-2,三种计算图

有三种计算图的构建方式：**静态计算图**，**动态计算图**，以及 **Autograph**.

在 TensorFlow 1.0 时代，采用的是静态计算图，需要先使用 TensorFlow 的各种算子创建计算图，然后再开启一个会话 Session，显式执行计算图。

而在 TensorFlow 2.0 时代，采用的是动态计算图，即每使用一个算子后，该算子会被动态加入到隐含的默认计算图中立即执行得到结果，而无需开启 Session。

使用动态计算图即 Eager Excution 的好处是方便调试程序，它会让 TensorFlow 代码的表现和 Python 原生代码的表现一样，写起来就像写 numpy 一样，各种日志打印，控制流全部都是可以使用的。

使用动态计算图的缺点是运行效率相对会低一些。因为使用动态图会有许多次 Python 进程和 TensorFlow 的 C++ 进程之间的通信。而静态计算图构建完成之后几乎全部在 TensorFlow 内核上使用 C++ 代码执行，效率更高。此外静态图会对计算步骤进行一定的优化，剪去和结果无关的计算步骤。

### 一，计算图简介

计算图由节点 (nodes) 和线 (edges) 组成。

节点表示操作符 Operator，或者称之为算子，线表示计算间的依赖。

实线表示有数据传递依赖，传递的数据即张量。

虚线通常可以表示控制依赖，即执行先后顺序。

![strjoin_graph](./data/strjoin_graph.png)

### 二，静态计算图

在 TensorFlow 1.0 中，使用静态计算图分两步，第一步**定义计算图**，第二步**在会话中执行计算图**。

TensorFlow 1.0 静态计算图范例

In [4]:
# import tensorflow as tf

# # 定义计算图
# g = tf.Graph()
# with g.as_default():
#     # placeholder 为占位符，执行会话时指定填充对象
#     x = tf.placeholder(name="x", shape=[], dtype=tf.string)
#     y = tf.placeholder(name="y", shape=[], dtype=tf.string)
#     z = tf.string_join([x, y], name="join", separator=" ")

**TensorFlow2.0 怀旧版静态计算图**

`TensorFlow 2.0` 为了确保对老版本 `tensorflow` 项目的兼容性，在 `tf.compat.v1` 子模块中保留了对 `TensorFlow 1.0` 那种静态计算图构建风格的支持。

可称之为怀旧版静态计算图，已经不推荐使用了。

In [10]:
import tensorflow as tf
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

g = tf.compat.v1.Graph()
with g.as_default():
    x = tf.compat.v1.placeholder(name="x", shape=[], dtype=tf.string)
    y = tf.compat.v1.placeholder(name="y", shape=[], dtype=tf.string)
    z = tf.strings.join([x, y], name="join", separator=" ")

with tf.compat.v1.Session(graph=g) as sess:
    result = sess.run(fetches=z, feed_dict={x: "hello", y: "world"})
    print(result)

b'hello world'


2022-05-06 12:40:25.109950: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 12:40:25.110272: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 12:40:25.110702: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 12:40:25.111186: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 12:40:25.112006: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

### 三，动态计算图

在 TensorFlow 2.0 中，使用的是动态计算图和 Autograph.

在 TensorFlow 1.0 中，使用静态计算图分两步，第一步定义计算图，第二步在会话中执行计算图。

动态计算图已经不区分计算图的定义和执行了，而是定义后立即执行。因此称之为 Eager Excution.

Eager 这个英文单词的原意是"迫不及待的"，也就是立即执行的意思。